<a href="https://colab.research.google.com/github/SESCDS/sescds/blob/master/Part%202/Lesson%205/Lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Импорт библиотек и установка random_seed - параметра, который обеспечит воспроизводимость случайных чисел
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import matplotlib.pyplot as plt
torch.manual_seed(1)

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
bs = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=bs,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=bs,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

100%|█████████▉| 170467328/170498071 [04:27<00:00, 536672.90it/s]

Files already downloaded and verified


In [0]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Linear(8*8*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

image_clf = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(image_clf.parameters(), lr=0.001, momentum=0.9)

In [4]:
def train(model, num_epochs):
  
  for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [0]:
def validate(model):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

In [6]:
btrain(image_clf, 4)

[1,    50] loss: 2.290
[1,   100] loss: 2.249
[1,   150] loss: 2.157
[1,   200] loss: 2.063
[1,   250] loss: 1.995
[1,   300] loss: 1.970
[1,   350] loss: 1.928
[1,   400] loss: 1.875
[1,   450] loss: 1.861
[1,   500] loss: 1.820
[1,   550] loss: 1.816
[1,   600] loss: 1.772
[1,   650] loss: 1.754
[1,   700] loss: 1.754
[1,   750] loss: 1.717
[2,    50] loss: 1.687
[2,   100] loss: 1.642
[2,   150] loss: 1.615
[2,   200] loss: 1.595
[2,   250] loss: 1.591
[2,   300] loss: 1.571
[2,   350] loss: 1.540
[2,   400] loss: 1.530
[2,   450] loss: 1.503
[2,   500] loss: 1.533
[2,   550] loss: 1.487
[2,   600] loss: 1.519
[2,   650] loss: 1.464
[2,   700] loss: 1.478
[2,   750] loss: 1.483
[3,    50] loss: 1.443
[3,   100] loss: 1.438
[3,   150] loss: 1.398
[3,   200] loss: 1.425
[3,   250] loss: 1.381
[3,   300] loss: 1.407
[3,   350] loss: 1.415
[3,   400] loss: 1.400
[3,   450] loss: 1.386
[3,   500] loss: 1.379
[3,   550] loss: 1.398
[3,   600] loss: 1.393
[3,   650] loss: 1.356
[3,   700] 

In [9]:
validate(image_clf)

Accuracy of the network on the 10000 test images: 54 %


<h1>Домашнее задание</h1>

Поэкспериментируйте с размерами фильтров и количеством слоев. Как меняется скорость и точность обучения в зависимости от этих параметров?